<h2>Processing the Modelling Data</h2>
<p>This file is concerned with processing and cleaning the ACS data</p>

In [ ]:
# Import necessary packages
import pandas as pd
# pd.options.mode.chained_assignment = None  # default='warn'
import os
import re
import glob
import numpy as np
from functools import reduce

<h3>Initial Data Processing</h3>
<p>All datasets can be initially processed to remove unnecessary columns and convert columns to numeric</p>

In [ ]:
def process_dfs(dfs_raw):
    
    for df in dfs_raw:
        # drop the first row of every dataset
        df.drop(index = df.index[0], axis = 0, inplace = True)
        # strip GEO_ID of first 11 characters
        df.loc[:,'GEO_ID'] = df.loc[:, 'GEO_ID'].str[-11:]
        # drop NAME column
        df.drop("NAME", axis=1, inplace=True)
        # drop all margin of error columns
        df.drop(list(df.filter(regex = 'M')), axis = 1, inplace = True)
        # convert non-GEO_ID columns to numeric
        num_cols = df.columns.drop('GEO_ID')
        df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')
        # drop all NA values
        df.dropna(inplace= True)
    
    return dfs_raw

<h3>Cleaning the Race Data</h3>

In [ ]:
def clean_acsRace(df):
    acs_race = df.copy()
    acs_race.rename(columns={'GEO_ID':'geoid',
                             'B02001_001E':'total', 
                             'B02001_002E':'white',
                             'B02001_003E':'black',
                             'B02001_004E':'nativeam',
                             'B02001_005E':'asian', 
                             'B02001_006E':'pacislander'}, inplace=True)
    
    acs_race['other'] = acs_race['B02001_007E'] + acs_race['B02001_008E']
    acs_race.drop(['B02001_007E', 
                   'B02001_008E', 
                   'B02001_009E', 
                   'B02001_010E'], axis=1, inplace=True)
    
    return acs_race

<h3>Cleaning the Age Data</h3>

In [ ]:
def clean_acsAge(df):
    acs_age = df.copy()
    acs_age.drop(['B01001_002E', 
                   'B01001_026E'], axis=1, inplace=True)
    acs_age.rename(columns={'GEO_ID':'geoid',
                             'B01001_001E':'total'}, inplace=True)
    
    acs_age['children'] = acs_age['B01001_003E'] + acs_age['B01001_004E'] + acs_age['B01001_005E'] + acs_age['B01001_006E'] + acs_age['B01001_027E'] + acs_age['B01001_028E'] + acs_age['B01001_029E'] + acs_age['B01001_030E']
    acs_age.drop(['B01001_003E', 
                  'B01001_004E', 
                  'B01001_005E', 
                  'B01001_006E', 
                  'B01001_027E', 
                  'B01001_028E', 
                  'B01001_029E', 
                  'B01001_030E'], axis=1, inplace=True)

    acs_age['youngAdult'] = acs_age['B01001_007E'] + acs_age['B01001_008E'] + acs_age['B01001_009E'] + acs_age['B01001_010E'] + acs_age['B01001_031E'] + acs_age['B01001_032E'] + acs_age['B01001_033E'] + acs_age['B01001_034E']
    acs_age.drop(['B01001_007E', 
                  'B01001_008E', 
                  'B01001_009E', 
                  'B01001_010E', 
                  'B01001_031E', 
                  'B01001_032E', 
                  'B01001_033E', 
                  'B01001_034E'], axis=1, inplace=True)

    acs_age['adult'] = acs_age['B01001_011E'] + acs_age['B01001_012E'] + acs_age['B01001_013E'] + acs_age['B01001_014E'] + acs_age['B01001_015E'] + acs_age['B01001_016E'] + acs_age['B01001_017E'] + acs_age['B01001_018E'] + acs_age['B01001_019E'] + acs_age['B01001_035E'] + acs_age['B01001_036E'] + acs_age['B01001_037E'] + acs_age['B01001_038E'] + acs_age['B01001_039E'] + acs_age['B01001_040E'] + acs_age['B01001_041E'] + acs_age['B01001_042E'] + acs_age['B01001_043E']
    acs_age.drop(['B01001_011E', 
                  'B01001_012E', 
                  'B01001_013E', 
                  'B01001_014E', 
                  'B01001_015E', 
                  'B01001_016E', 
                  'B01001_017E', 
                  'B01001_018E',
                  'B01001_019E', 
                  'B01001_035E', 
                  'B01001_036E', 
                  'B01001_037E', 
                  'B01001_038E', 
                  'B01001_039E', 
                  'B01001_040E', 
                  'B01001_041E', 
                  'B01001_042E',
                  'B01001_043E'], axis=1, inplace=True)

    acs_age['elderly'] = acs_age['B01001_020E'] + acs_age['B01001_021E'] + acs_age['B01001_022E'] + acs_age['B01001_023E'] + acs_age['B01001_024E'] + acs_age['B01001_025E'] + acs_age['B01001_044E'] + acs_age['B01001_045E'] + acs_age['B01001_046E'] + acs_age['B01001_047E'] + acs_age['B01001_048E'] + acs_age['B01001_049E']
    acs_age.drop(['B01001_020E', 
                  'B01001_021E',  
                  'B01001_022E', 
                  'B01001_023E', 
                  'B01001_024E', 
                  'B01001_025E', 
                  'B01001_044E', 
                  'B01001_045E', 
                  'B01001_046E', 
                  'B01001_047E', 
                  'B01001_048E', 
                  'B01001_049E'], axis=1, inplace=True)

    
    return acs_age

<h3>Cleaning the Tenure Data</h3>

In [ ]:
def clean_acsTenure(df):
    
    acs_tenure = df.copy()
    acs_tenure.rename(columns={'GEO_ID':'geoid', 
                               'B25003_001E':'total', 
                               'B25003_002E':'ownerOcc', 
                               'B25003_003E':'renterOcc'}, inplace=True)
    
    return acs_tenure

<h3>Cleaning the Hours Worked Data</h3>

In [ ]:
def clean_acsHoursWk(df):
    
    acs_hoursWk = df.copy()
    acs_hoursWk.drop(['B23020_002E', 
                  'B23020_003E'], axis=1, inplace=True)
    acs_hoursWk.rename(columns={'GEO_ID':'geoid', 
                                'B23020_001E':'hoursWk'}, inplace=True)
    
    return acs_hoursWk

<h3>Cleaning the Citizenship Data</h3>

In [ ]:
def clean_acsCitizen(df):
    
    acs_citizen = df.copy()
    acs_citizen.rename(columns={'GEO_ID':'geoid', 
                                'B05001_001E':'total',
                                'B05001_002E':'citizenUS',
                                'B05001_003E':'citizenPR',
                                'B05001_004E':'citizenAbroad',
                                'B05001_005E':'citizenNaturalized',
                                'B05001_006E':'notCitizen'}, inplace=True)
    
    return acs_citizen

<h3>Cleaning the Marital Status Data</h3>

In [ ]:
def clean_acsMarriage(df):
    
    acs_marriage = df.copy()
    acs_marriage.drop(['B12001_002E', 
                      'B12001_011E',
                      'B12001_004E',
                      'B12001_006E',
                      'B12001_013E',
                      'B12001_015E'], axis=1, inplace=True)
    acs_marriage.rename(columns={'GEO_ID':'geoid', 
                                'B12001_001E':'total'}, inplace=True)
    
    acs_marriage['single'] = acs_marriage['B12001_003E'] + acs_marriage['B12001_012E']
    acs_marriage.drop(['B12001_003E', 'B12001_012E'], axis=1, inplace=True)

    acs_marriage['married'] = acs_marriage['B12001_005E'] + acs_marriage['B12001_014E']
    acs_marriage.drop(['B12001_005E', 'B12001_014E'], axis=1, inplace=True)

    acs_marriage['separated'] = acs_marriage['B12001_007E'] + acs_marriage['B12001_016E']
    acs_marriage.drop(['B12001_007E', 'B12001_016E'], axis=1, inplace=True)

    acs_marriage['marriedOther'] = acs_marriage['B12001_008E'] + acs_marriage['B12001_017E']
    acs_marriage.drop(['B12001_008E', 'B12001_017E'], axis=1, inplace=True)

    acs_marriage['widowed'] = acs_marriage['B12001_009E'] + acs_marriage['B12001_018E']
    acs_marriage.drop(['B12001_009E', 'B12001_018E'], axis=1, inplace=True)

    acs_marriage['divorced'] = acs_marriage['B12001_010E'] + acs_marriage['B12001_019E']
    acs_marriage.drop(['B12001_010E', 'B12001_019E'], axis=1, inplace=True)
    
    return acs_marriage

<h3>Cleaning the Transportation Data</h3>

In [ ]:
def clean_acsTrans(df):
    
    acs_trans = df.copy()
    acs_trans.drop(['B08301_002E', 
                    'B08301_010E',
                    'B08301_004E'], axis=1, inplace=True)
    acs_trans.rename(columns={'GEO_ID':'geoid', 
                              'B08301_001E':'total', 
                              'B08301_011E':'bus', 
                              'B08301_014E':'train', 
                              'B08301_015E':'ferry', 
                              'B08301_016E':'taxi', 
                              'B08301_018E':'bicycle', 
                              'B08301_019E':'walked', 
                              'B08301_020E':'otherTravel', 
                              'B08301_021E':'homeworker'}, inplace=True)
    
    acs_trans['privateVehicle'] = acs_trans['B08301_003E'] + acs_trans['B08301_017E']
    acs_trans.drop(['B08301_003E', 'B08301_017E'], axis=1, inplace=True)

    acs_trans['carpool'] = acs_trans['B08301_005E'] + acs_trans['B08301_006E'] + acs_trans['B08301_007E'] + acs_trans['B08301_008E'] + acs_trans['B08301_009E']
    acs_trans.drop(['B08301_005E', 'B08301_006E', 'B08301_007E', 'B08301_008E', 'B08301_009E'], axis=1, inplace=True)

    acs_trans['subwayTram'] = acs_trans['B08301_012E'] + acs_trans['B08301_013E']
    acs_trans.drop(['B08301_012E', 'B08301_013E'], axis=1, inplace=True)
    
    return acs_trans

<h3>Cleaning Structure Build Data</h3>

In [ ]:
def clean_acsYearBuilt_2010(df):
    
    acs_yearBuilt = df.copy()
    
    acs_yearBuilt.rename(columns={'GEO_ID':'geoid', 
                                  'B25034_001E':'total',  
                                  'B25034_004E':'1990_1999', 
                                  'B25034_005E':'1980_1989', 
                                  'B25034_006E':'1970_1979', 
                                  'B25034_007E':'1960_1969', 
                                  'B25034_008E':'1950_1959', 
                                  'B25034_009E':'1940_1949',
                                  'B25034_010E':'pre_1939'}, inplace=True)
    
    acs_yearBuilt['2000_later'] = acs_yearBuilt['B25034_002E'] + acs_yearBuilt['B25034_003E']
    acs_yearBuilt.drop(['B25034_002E', 'B25034_003E'], axis=1, inplace=True)
    
    return acs_yearBuilt

In [ ]:
def clean_acsYearBuilt_2019(df):
    
    acs_yearBuilt = df.copy()
    
    acs_yearBuilt.rename(columns={'GEO_ID':'geoid', 
                                  'B25034_001E':'total',   
                                  'B25034_005E':'1990_1999', 
                                  'B25034_006E':'1980_1989', 
                                  'B25034_007E':'1970_1979', 
                                  'B25034_008E':'1960_1969', 
                                  'B25034_009E':'1950_1959', 
                                  'B25034_010E':'1940_1949',
                                  'B25034_011E':'pre_1939'}, inplace=True)
    
    acs_yearBuilt['2000_later'] = acs_yearBuilt['B25034_002E'] + acs_yearBuilt['B25034_003E'] + acs_yearBuilt['B25034_004E']
    acs_yearBuilt.drop(['B25034_002E', 'B25034_003E', 'B25034_004E'], axis=1, inplace=True)
    
    return acs_yearBuilt

<h3>Cleaning Units in Structure Data</h3>

In [ ]:
def clean_acsStructure(df):
    
    acs_structure = df.copy()
    
    acs_structure.rename(columns={'GEO_ID':'geoid', 
                                  'B25024_001E':'total', 
                                  'B25024_002E':'detatched_1', 
                                  'B25024_003E':'attached_1', 
                                  'B25024_004E':'2_unit', 
                                  'B25024_005E':'3to4_unit', 
                                  'B25024_006E':'5to9_unit', 
                                  'B25024_007E':'10to19_unit', 
                                  'B25024_008E':'20to49_unit', 
                                  'B25024_009E':'50plus_unit'}, inplace=True)
    
    acs_structure['mobile'] = acs_structure['B25024_010E'] + acs_structure['B25024_011E']
    acs_structure.drop(['B25024_010E', 'B25024_011E'], axis=1, inplace=True)
    
    return acs_structure

<h3>Convert Necessary Values to Shares</h3>

In [ ]:
def toShare(df):
    pred_data = pd.DataFrame()
    for n in df.columns.tolist():
        #print("Converting " + n)
        if n == 'geoid':
            pred_data.loc[:,n] = df.loc[:,'geoid']
        else:
            pred_data.loc[:,n] = (df.loc[:,n] / df.loc[:,'total'])
            
    pred_data.drop(['total'], axis=1, inplace=True)
    
    return pred_data

<h3>Creating the Dataset</h3>
<p>Read in the data, clean each dataset individually, include certain datasets in the convert to percentages (or convert to shares) process, merge all datasets together by common geo_id column</p>

In [ ]:
def process_data(df_list, year):
    
    # Perform initial cleaning for all data
    print("perform initial processing...")
    clean_dfs = process_dfs(dataframes_list)
    
    # Clean each dataset appropriately
    acs_race = clean_acsRace(clean_dfs[0])
    print("cleaning race data...")

    acs_age = clean_acsAge(clean_dfs[1])
    print("cleaning age data...")

    acs_tenure = clean_acsTenure(clean_dfs[2])
    print("cleaning tenure data...")

    acs_hoursWk = clean_acsHoursWk(clean_dfs[3])
    print("cleaning hours worked data...")

    acs_citizen = clean_acsCitizen(clean_dfs[4])
    print("cleaning citizenship data...")

    acs_marriage = clean_acsMarriage(clean_dfs[5])
    print("cleaning marriage data...")

    acs_trans = clean_acsTrans(clean_dfs[6])
    print("cleaning transportation data...")
    
    if year == 2010:
        acs_yearBuilt = clean_acsYearBuilt_2010(clean_dfs[7])
        print("cleaning year built data...")
    else:
        acs_yearBuilt = clean_acsYearBuilt_2019(clean_dfs[7])
        print("cleaning year built data...")
    
    acs_structure = clean_acsStructure(clean_dfs[8])
    print("cleaning structure data...")

    # Convert necessary datasets to shares
    to_share = [acs_race, acs_age, acs_tenure, acs_citizen, acs_marriage, acs_trans, acs_yearBuilt, acs_structure]
    percents = []
    
    print("create shares...")
    for df in to_share:
        temp = df.copy()
        percents.append(toShare(temp))
    
    # Combine all datasets into one
    print("merging datasets...")
    df_all = percents + [acs_hoursWk]
    df_merged = reduce(lambda x,y: pd.merge(x,y, on='geoid', how='outer'), df_all)   
    
    df_merged.set_index('geoid', inplace=True)
    df_merged.fillna(0, inplace=True)

    return df_merged

In [ ]:
# In the main function

# Process the 2010 Data
print("processing 2010 data...")
list_of_names = ['ACSDT5Y2010.B02001', #0 Race
                 'ACSDT5Y2010.B01001', #1 Age
                 'ACSDT5Y2010.B25003', #2 Tenure
                 'ACSDT5Y2010.B23020', #3 Hours worked
                 'ACSDT5Y2010.B05001', #4 Citizenship
                 'ACSDT5Y2010.B12001', #5 Marital Status
                 'ACSDT5Y2010.B08301', #6 Transportation
                 'ACSDT5Y2010.B25034', #7 Year Structure Built
                 'ACSDT5Y2010.B25024'] #8 Units in Structure
# create empty list
dataframes_list = []
 
# append datasets into the list
for i in range(len(list_of_names)):
    temp_df = pd.read_csv("/Users/ritalaplante/Desktop/Thesis Data and Analytics/01-Raw Data/ACS_2010/"+list_of_names[i]+".csv")
    dataframes_list.append(temp_df)
    
data_2010 = process_data(dataframes_list, 2010)
print("dataset creation complete...\n")

# Process the 2019 Data
print("processing 2019 data...")
list_of_names = ['ACSDT5Y2019.B02001', #0 Race
                 'ACSDT5Y2019.B01001', #1 Age
                 'ACSDT5Y2019.B25003', #2 Tenure
                 'ACSDT5Y2019.B23020', #3 Hours worked
                 'ACSDT5Y2019.B05001', #4 Citizenship
                 'ACSDT5Y2019.B12001', #5 Marital Status
                 'ACSDT5Y2019.B08301', #6 Transportation
                 'ACSDT5Y2019.B25034', #7 Year Structure Built
                 'ACSDT5Y2019.B25024'] #8 Units in Structure
# create empty list
dataframes_list = []
 
# append datasets into the list
for i in range(len(list_of_names)):
    temp_df = pd.read_csv("/Users/ritalaplante/Desktop/Thesis Data and Analytics/01-Raw Data/ACS_2019/"+list_of_names[i]+".csv")
    dataframes_list.append(temp_df)
    
data_2019 = process_data(dataframes_list, 2019)
print("dataset creation complete...")


In [ ]:
data_2010.head()

In [ ]:
data_2019.head()

In [ ]:
s19 = data_2019.columns
s10 = data_2010.columns
print("2019 variables diff'd against 2010 variables: " + str(s19.difference(s10)))
print("2010 variables diff'd against 2019 variables: " + str(s10.difference(s19)))

In [ ]:
#os.makedirs('/Users/ritalaplante/Desktop/Senior Thesis/Data/outputs', exist_ok=True)  
data_2010.to_csv('/Users/ritalaplante/Desktop/Thesis Data and Analytics/02-Cleaned Predictor Data/predictor2010.csv', index = True)  
data_2019.to_csv('/Users/ritalaplante/Desktop/Thesis Data and Analytics/02-Cleaned Predictor Data/predictor2019.csv', index = True)  